In [18]:
import csv
import pickle
import numpy as np

fail_file = 'fail'
with open(fail_file, 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    fail_list = list(reader)
    fail_id = [x[0] for x in fail_list]
    fail_id = [int(x.replace('qid: ', '')) for x in fail_id]
    # print(fail_id)

qa_path = '/root/.cache/anserini/topics-and-qrels/'
qa_file = f'{qa_path}topics.dpr.nq.train.txt'

with open(qa_file, 'r', encoding='utf-8') as file:
    data = [line.split('\t')[0] for line in file]

questions = [item for item in data]
# print(questions[:5])


cutoff = 200
pkl_file = f'/dk/lcj_rag/pyserini-autofusion/experiments/runs/nq_train/nq_train_Xy-top{cutoff}.pkl'
# # Load data using pickle
with open(pkl_file, 'rb') as f:
    Xy_np = pickle.load(f)
    X_np = Xy_np[:, :-1]
    X_np = np.hstack((X_np[:, 768:868], X_np[:, 968:1068]))
    print(X_np.shape)


import ast

alpha_file = '/dk/lcj_rag/pyserini-autofusion/experiments/runs/nq_test/run.dpr.nq-test.dpr.bm25.fusion.gt.all-alpha.nq--1.k-100.res.test'
class0 = []
class1 = []
class2 = []
class0_decay0 = []
class0_decay1 = []
class1_decay0 = []
class1_decay1 = []
class2_decay0 = []
class2_decay1 = []

last = 100

num_fail = len(fail_id)
with open(alpha_file, 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    lines = list(reader)
    # (qid, hits, softmax_hits, weights)
    for i in range(len(lines)):
        hits = lines[i][1]
        hits = hits.replace('hits: ', '')
        # hits = hits.to_list()
        hits = ast.literal_eval(hits)
        if i in fail_id and sum(hits):
            if hits[0] and hits[-1] == 0:
                # class0 += 1
                class0.append((i, hits))
                class0_decay0.append(X_np[i, last - 1] / X_np[i, 0])
                class0_decay1.append(X_np[i, 100 + last - 1] / X_np[i, 100])
            if hits[0] == 0 and hits[-1]:
                # class1 += 1
                class1.append((i, hits))
                class1_decay0.append(X_np[i, last - 1] / X_np[i, 0])
                class1_decay1.append(X_np[i, 100 + last - 1] / X_np[i, 100])
        else:
            class2.append((i, hits))
            class2_decay0.append(X_np[i, last - 1] / X_np[i, 0])
            class2_decay1.append(X_np[i, 100 + last - 1] / X_np[i, 100])


print(f'num_class0: {len(class0)}')
for i, (qid, hits) in enumerate(class0):
    print(f'qid: {qid}\t| decay0: {class0_decay0[i]} | decay1: {class0_decay1[i]} | score0: {X_np[qid, :5]} | score1: {X_np[qid, 105:105 + 5]}\t| question: {questions[qid]}')
print(f'num_class1: {len(class1)}')
for i, (qid, hits) in enumerate(class1):
    print(f'qid: {qid}\t| decay0: {class1_decay0[i]} | decay1: {class1_decay1[i]} | score0: {X_np[qid, :5]} | score1: {X_np[qid, 105:105 + 5]}\t| question: {questions[qid]}')

# print(f'avg: class0_decay0: {np.mean(class0_decay0)}')
print(f'avg: class0_decay0: {np.mean(class0_decay0)}')
print(f'std: class0_decay0: {np.std(class0_decay0)}')
# print(f'avg: class1_decay0: {np.mean(class1_decay0)}')
print(f'avg: class1_decay0: {np.mean(class1_decay0)}')
print(f'std: class1_decay0: {np.std(class1_decay0)}')

print(f'avg: class2_decay0: {np.mean(class2_decay0)}')
print(f'std: class2_decay0: {np.std(class2_decay0)}')

print(f'class2_decay0 - class0_decay0: {np.mean(class2_decay0) - np.mean(class0_decay0)}')
print(1 - (num_fail - 0) / 3610)

# last  delta_decay
# 5     0.01038106804426775
# 10    0.011653730710922572
# 25    0.012135627917595926
# 50    0.012989433003550044
# 100   0.013276806527493745

(79168, 200)
num_class0: 54
qid: 151	| decay0: 0.8721233511778377 | decay1: 0.6384814291838105 | score0: [81.285904 79.161728 75.363266 74.900536 74.512642] | score1: [11.456995 11.398537 11.151223 10.630761 10.602483]	| question: who starred in the secret life of bees
qid: 241	| decay0: 0.9010463474684729 | decay1: 0.7500014336053845 | score0: [86.239899 86.092903 85.51564  84.666794 84.077896] | score1: [13.689622 13.540649 13.355507 13.298246 13.228098]	| question: who commanded british forces at the battle of waterloo
qid: 295	| decay0: 0.8221300516285467 | decay1: 0.6680458089463643 | score0: [90.875694 84.842865 83.729317 83.339005 83.230103] | score1: [12.084929 11.887501 11.887501 11.471573 11.410019]	| question: when does the next episode of superstore air
qid: 389	| decay0: 0.8882954178327016 | decay1: 0.6454698033357102 | score0: [81.34404  79.353798 78.527725 78.005295 77.567169] | score1: [24.609959 24.570299 24.050491 23.633095 23.533642]	| question: where did they shoot 

In [1]:
import csv
import pickle
import numpy as np

fail_file = 'fail'
with open(fail_file, 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    fail_list = list(reader)
    fail_id = [x[0] for x in fail_list]
    fail_id = [int(x.replace('qid: ', '')) for x in fail_id]
    # print(fail_id)

qa_path = '/root/.cache/anserini/topics-and-qrels/'
qa_file = f'{qa_path}topics.dpr.nq.train.txt'

with open(qa_file, 'r', encoding='utf-8') as file:
    data = [line.split('\t')[0] for line in file]

questions = [item for item in data]
# print(questions[:5])


cutoff = 200
pkl_file = f'/dk/lcj_rag/pyserini-autofusion/experiments/runs/nq_train/nq_train_Xy-top{cutoff}.pkl'
# # Load data using pickle
with open(pkl_file, 'rb') as f:
    Xy_np = pickle.load(f)
    X_np = Xy_np[:, :-1]
    X_np = np.hstack((X_np[:, 768:868], X_np[:, 968:1068]))
    print(X_np.shape)


import ast

alpha_file = '/dk/lcj_rag/pyserini-autofusion/experiments/runs/nq_test/run.dpr.nq-test.dpr.bm25.fusion.gt.all-alpha.nq--1.k-100.res.test'
class0 = []
class1 = []
class2 = []
class0_dist = []
class1_dist = []
class2_dist = []

def weighted_distance(np_x):
    w = np.array([1 / x for x in np.arange(1, len(np_x) + 1)])
    # w = np.array([np.exp(-x) for x in np.arange(1, len(np_x) + 1)])
    # w = w / np.sum(w)
    # print(w)
    return np.dot(np_x, w)

# weighted_distance(X_np[0, :10])

num_fail = len(fail_id)
with open(alpha_file, 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    lines = list(reader)
    # (qid, hits, softmax_hits, weights)
    for i in range(len(lines)):
        hits = lines[i][1]
        hits = hits.replace('hits: ', '')
        # hits = hits.to_list()
        hits = ast.literal_eval(hits)
        if i in fail_id and sum(hits):
            if hits[0] and hits[-1] == 0:
                class0.append((i, hits))
                class0_dist.append(weighted_distance(X_np[i, :100]))
            if hits[0] == 0 and hits[-1]:
                class1.append((i, hits))
                class1_dist.append(weighted_distance(X_np[i, :100]))
        else:
            class2.append((i, hits))
            class2_dist.append(weighted_distance(X_np[i, :100]))

print(f'avg: class0_dist: {np.mean(class0_dist)}')
print(f'minmax: class0_dist: {np.min(class0_dist), np.max(class0_dist)}')
print(f'std: class0_dist: {np.std(class0_dist)}')

print(f'avg: class1_dist: {np.mean(class1_dist)}')
print(f'minmax: class1_dist: {np.min(class1_dist), np.max(class1_dist)}')
print(f'std: class1_dist: {np.std(class1_dist)}')

print(f'avg: class2_dist: {np.mean(class2_dist)}')
print(f'minmax: class2_dist: {np.min(class2_dist), np.max(class2_dist)}')
print(f'std: class2_dist: {np.std(class2_dist)}')

print(f'class2_dist - class0_dist: {np.mean(class2_dist) - np.mean(class0_dist)}')
print(1 - (num_fail - 0) / 3610)


# draw hist of the 3 classes in the same fig
import matplotlib.pyplot as plt

class0_dist = np.array(class0_dist)
class1_dist = np.array(class1_dist)
class2_dist = np.array(class2_dist)

threshold = 440
print(np.count_nonzero(class0_dist > threshold) / len(class0_dist))
print(np.count_nonzero(class2_dist > threshold) / len(class2_dist))
# plt.hist(class0_dist, bins=100, alpha=0.5, label='class0')
# plt.hist(class1_dist, bins=100, alpha=0.5, label='class1')
# plt.hist(class2_dist, bins=100, alpha=0.5, label='class2')

# last  delta_decay
# 5     0.01038106804426775
# 10    0.011653730710922572
# 25    0.012135627917595926
# 50    0.012989433003550044
# 100   0.013276806527493745
import csv
with open('class1.csv', 'w') as f:
    writer = csv.writer(f, delimiter='\t')
    # class1.append((i, hits))
    for id, _ in class1:
        writer.writerow([id])

(79168, 200)
avg: class0_dist: 408.51572515838154
minmax: class0_dist: (384.10597674908587, 449.62578191125283)
std: class0_dist: 16.592871571817046
avg: class1_dist: 408.48091321703555
minmax: class1_dist: (357.9730936540601, 433.44478799119844)
std: class1_dist: 21.2163260352651
avg: class2_dist: 409.9711247098188
minmax: class2_dist: (344.6582148219528, 505.3934076306393)
std: class2_dist: 20.849851552578965
class2_dist - class0_dist: 1.4553995514372673
0.8806094182825485
0.037037037037037035
0.07979626485568761
